Lab2

Initializing modules we`ll need for this lab

In [1]:
import pandas as pd
import urllib.request
import datetime
import os
import shutil
from IPython.display import display

Initializing dictionary with old and new province indexes

In [2]:
province_index_dict = {
    1: (22, 'Черкаська'), 2: (24, 'Чернігівська'), 3: (23, 'Чернівецька'), 4: (25, 'Крим'), 5: (3, 'Дніпропетровська'), 
    6: (4, 'Донецька'), 7: (8, 'Івано-Франківська'), 8: (19 , 'Харківська'), 9: (20, 'Херсонська'), 10: (21, 'Хмельницька'),
    11: (9, 'Київська'), 13: (10, 'Кіровоградська'), 14: (11, 'Луганська'), 15: (12, 'Львівська'), 16: (13, 'Миколаївська'), 
    17: (14, 'Одеська'), 18: (15, 'Полтавська'), 19: (16, 'Рівненська'), 21: (17, 'Сумська'), 22: (18, 'Тернопільська'),
    23: (6, 'Закарпатська'), 24: (1, 'Вінницька'), 25: (2, 'Волинська'), 26: (7, 'Запорізька'), 27: (5, 'Житомирська')
    }

Initializing headers for the dataframe and array with dataframes for each province separately

In [3]:
headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
dfs = []

Methods to get urls with the datasets to analyze and to create csv files with the corresponding datasets

In [4]:
def get_url(i: int) -> str:
    return f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean'

def file_create(i: int, data: bytes, csv_path: str) -> None:
    date_time = datetime.datetime.now().strftime("%d%m%Y%H%M%S")
    out = open(f'{csv_path}/vhi_id_{i}_{date_time}.csv', "wb")
    out.write(data)


Methods to create dataframes with datasets from each file and to normalize dataframes by deleting NaN (-1.0) values, changing data types, removing columns 'empty' and adding new columns 'ProvinceID'. Also each dataframe is added to array with dataframes

In [5]:
def dataframe_create(path: str) -> None:    
    for file in (os.listdir(path)):
        i = file.split(sep='_')[2]
        df = pd.read_csv(f'{path}/{file}', header = 1, names = headers)
        normalize_dataframe(df, i)


def normalize_dataframe(df: pd.DataFrame, i: int) -> None:
    # dropping NaN values
    df = df.dropna(subset=['VHI']).drop(columns=['empty'])
    df = df.drop(df.loc[df['VHI'] == -1].index)
    # normalizing data
    df["Year"] = df["Year"].str.extract(r"(\d+)").astype(int)
    df['Week'] = df['Week'].astype(int)

    #Adding ids for all provinces
    df.insert(0, 'ProvinceID', [int(i)] * df.shape[0])
   
    dfs.append(df)


Metho to add column with provinces' names to the general dataframe based on their English indexes

In [6]:
def add_province_names(general_df: pd.DataFrame) -> None:
    for key, value in province_index_dict.items():
        general_df.loc[general_df['ProvinceID'] == key, 'ProvinceName'] = value[1]
        
    province_column = general_df.pop('ProvinceName')
    general_df.insert(1, 'ProvinceName', province_column)

Method to reorder(change) provinces' indexes from English alphabet to Ukrainian

In [7]:
def change_ProvinceID(general_df: pd.DataFrame) -> None:
    replace_dict = {key: value[0] for key, value in province_index_dict.items()}
    general_df['ProvinceID'] = general_df['ProvinceID'].replace(replace_dict)


Method to display VHIs for the specific province in the specific year

In [8]:
def show_one_province_vhi(general_df: pd.DataFrame, year: int, index: int) -> None:
    if year not in range(1982, 2025):
        print('Invalid year for calculating')
        return
    elif index not in range(1, 26):
        print('No province with such index')
        return
    

    display(general_df[(general_df["ProvinceID"] == index) 
                       & (general_df["Year"] == year)][['ProvinceName', 'Year', 'Week','VHI']])

Method to find extremums, mean and median of any column values for specific provinces(can be 2 or more) at the specific period of time

In [9]:
def find_extremums(general_df: pd.DataFrame, province_names: list, years: tuple, value_to_seek: str) -> None:
    if years[0]<1982 or years[1]>2024 or years[0]>years[1]:
        print('Invalid years for calculating')
        return
    elif any(province not in set(general_df['ProvinceName']) for province in province_names):
        print('No such province in the dataframe')
        return
    elif value_to_seek not in set(general_df.columns):
        print(f'No such column in the dataframe: {value_to_seek}')
        return

    min_v: float = general_df[(general_df['Year']>=years[0]) & 
                              (general_df['Year']<=years[1]) & 
                              (general_df['ProvinceName'].isin(province_names))][value_to_seek].min()
    max_v: float = general_df[(general_df['Year']>=years[0]) & 
                              (general_df['Year']<=years[1]) & 
                              (general_df['ProvinceName'].isin(province_names))][value_to_seek].max()
    
    mean_v: float = general_df[(general_df['Year']>=years[0]) & 
                              (general_df['Year']<=years[1]) & 
                              (general_df['ProvinceName'].isin(province_names))][value_to_seek].mean().round(2)
    
    median_v: float = general_df[(general_df['Year']>=years[0]) & 
                              (general_df['Year']<=years[1]) & 
                              (general_df['ProvinceName'].isin(province_names))][value_to_seek].median().round(2)
    
    print(f'Minimum value of {value_to_seek} for {', '.join(province_names)} province(s) between {years[0]} and {years[1]}: {min_v}')
    print(f'Maximum value of {value_to_seek} for {', '.join(province_names)} province(s) between {years[0]} and {years[1]}: {max_v}')
    print(f'Mean value of {value_to_seek} for {', '.join(province_names)} province(s) between {years[0]} and {years[1]}: {mean_v}')
    print(f'Median value of {value_to_seek} for {', '.join(province_names)} province(s) between {years[0]} and {years[1]}: {median_v}')
    

Method to display VHIs for specific provinces(can be 2 or more) at the period of time

In [10]:
def show_multiple_province_vhis(general_df: pd.DataFrame, years: tuple, province_names: list) -> None:
    if years[0]<1982 or years[1]>2024 or years[0]>years[1]:
        print('Invalid years for calculating')
        return
    elif any(province not in set(general_df['ProvinceName']) for province in province_names):
        print('No such province in the dataframe')
        return
    
    display(general_df[(general_df['ProvinceName'].isin(province_names)) 
                       & (general_df['Year']>=years[0]) 
                       & (general_df['Year']<=years[1])][['ProvinceName', 'Year', 'Week','VHI']])

In [11]:
def show_draughts(general_df: pd.DataFrame) -> None: 
    drought_df = general_df[general_df['VHI']<=15]

    drought_count = drought_df.groupby('Year')['ProvinceName'].nunique()
    
    years = drought_count[drought_count>=5].index.tolist()

    display(general_df[general_df['Year'].isin(years) & (general_df['VHI']<=15)][['ProvinceName', 'Year', 'Week', 'VHI']])

Removing directory with csv files if it exists, creating new one with csv files

In [12]:
if os.path.isdir(csv_path:="csv_files"):
    shutil.rmtree(csv_path)

os.mkdir(csv_path)

#Creating files
for i in range(1, 28):
    if i == 12 or i == 20:
        continue
    else:
        url = get_url(i)
        data = urllib.request.urlopen(url).read()
        file_create(i, data, csv_path)
print("Files with data created")


Files with data created


Creating dataframes, normalizing them and merging all in one general dataframe, adding provinces' names, changing provinces' indexes and displaying it

In [13]:
#Creating separate dataframes for each province
dataframe_create(csv_path)

#Concatenating all dataframes into one general dataframe 
general_df = pd.concat(dfs, ignore_index=True)

#Adding provinces' names
add_province_names(general_df)

#Changing provinces' IDs
change_ProvinceID(general_df)

print('General normalized dataframe:')
display(general_df)


General normalized dataframe:


,ProvinceID,ProvinceName,Year,Week,SMN,SMT,VCI,TCI,VHI
0,21,Хмельницька,1982,1,0.059,258.24,51.11,48.78,49.95
1,21,Хмельницька,1982,2,0.063,261.53,55.89,38.20,47.04
2,21,Хмельницька,1982,3,0.063,263.45,57.30,32.69,44.99
3,21,Хмельницька,1982,4,0.061,265.10,53.96,28.62,41.29
4,21,Хмельницька,1982,5,0.058,266.42,46.87,28.57,37.72
...,...,...,...,...,...,...,...,...,...
54645,20,Херсонська,2024,48,0.135,278.17,55.23,11.86,33.55
54646,20,Херсонська,2024,49,0.133,277.08,57.71,10.86,34.29
54647,20,Херсонська,2024,50,0.130,276.49,59.45,8.68,34.07
54648,20,Херсонська,2024,51,0.128,276.45,62.53,5.55,34.04


Testing methods to display selected data

In [14]:
print('VHI for specific province:')
show_one_province_vhi(general_df, 1984, 3)

VHI for specific province:


,ProvinceName,Year,Week,VHI
43824,Дніпропетровська,1984,1,57.65
43825,Дніпропетровська,1984,2,59.34
43826,Дніпропетровська,1984,3,58.34
43827,Дніпропетровська,1984,4,54.79
43828,Дніпропетровська,1984,5,50.51
43829,Дніпропетровська,1984,6,47.89
43830,Дніпропетровська,1984,7,46.18
43831,Дніпропетровська,1984,8,46.60
43832,Дніпропетровська,1984,9,46.18
43833,Дніпропетровська,1984,10,44.28


In [15]:
find_extremums(general_df, ['Вінницька'], (1987, 1987), 'VHI')

Minimum value of VHI for Вінницька province(s) between 1987 and 1987: 21.06
Maximum value of VHI for Вінницька province(s) between 1987 and 1987: 62.61
Mean value of VHI for Вінницька province(s) between 1987 and 1987: 45.27
Median value of VHI for Вінницька province(s) between 1987 and 1987: 44.76


In [16]:
print('VHIs for different provinces at the certain period of time:')
show_multiple_province_vhis(general_df, (1987, 1999), ['Закарпатська', 'Донецька'])

VHIs for different provinces at the certain period of time:


,ProvinceName,Year,Week,VHI
26481,Закарпатська,1987,1,42.32
26482,Закарпатська,1987,2,43.64
26483,Закарпатська,1987,3,43.18
26484,Закарпатська,1987,4,41.41
26485,Закарпатська,1987,5,40.46
...,...,...,...,...
46807,Донецька,1999,48,37.51
46808,Донецька,1999,49,34.97
46809,Донецька,1999,50,30.26
46810,Донецька,1999,51,28.90


In [17]:
print('The year(s) that more than 20 percent of provinces had extreme droughts in:')
show_draughts(general_df)

The year(s) that more than 20 percent of provinces had extreme droughts in:


,ProvinceName,Year,Week,VHI
12201,Миколаївська,2007,22,11.44
12202,Миколаївська,2007,23,7.78
12203,Миколаївська,2007,24,6.44
12204,Миколаївська,2007,25,6.12
12205,Миколаївська,2007,26,6.11
12206,Миколаївська,2007,27,5.94
12207,Миколаївська,2007,28,6.36
12208,Миколаївська,2007,29,7.61
12209,Миколаївська,2007,30,9.45
12210,Миколаївська,2007,31,11.70
